In [1]:
'''
Program used to clustering news articles to find trends and patterns in the data.
We are using data from reddit API.
'''

'\nProgram used to clustering news articles to find trends and patterns in the data.\nWe are using data from reddit API.\n'

In [2]:
# Verifying reddit application
CLIENT_ID = "A8sawF4EfvDBEQ"
CLIENT_SECRET = "bvCCNdnlMNtUvn_6qLznm4uonHw"
USER_AGENT = "python: <rmartinsdev-data_mining> (by /u/rmartinsdev)"

In [3]:
# Creating username and password
from getpass import getpass

USERNAME = "rmartinsdev"
PASSWORD = "##########"

In [4]:
# Defining a function to log with this information
import requests

def login(username, password):
    if password is None:
        password = getpass.getpass(f"Enter reddit password for user {username}: ")
    headers = {"User-Agent": USER_AGENT}
    # Setup an auth object with our credentials
    client_auth = requests.auth.HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
    # Make a post request to the access_token endpoint
    post_data = {"grant_type": "password", "username": username, "password": password}
    response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)
    return response.json()

In [5]:
# Calling the function to get an access token
token = login(USERNAME, PASSWORD)
print(token)

{'access_token': '337851273076-Lo6gy6qsTV-CvHwjb4EjM5DZxSw', 'token_type': 'bearer', 'expires_in': 3600, 'scope': '*'}


In [6]:
# Obtaining sets of links from a subreddit
subreddit = "worldnews"
url = f"https://oauth.reddit.com/r/{subreddit}"

In [7]:
# Setting the headers
headers = {"Authorization": f"bearer {token['access_token']}", "User-Agent": USER_AGENT} 

In [8]:
# Requesting library to make the call, ensuring that we set the headers
response = requests.get(url, headers=headers)

In [9]:
# Storing the data in a dictionary and printing the titles of stories
result = response.json()
for story in result['data']['children']:
    print(story['data']['title'])

Livethread X: Global COVID-19 Pandemic
The UK rejects Trump's offer to help with Boris Johnson's coronavirus treatment
UK thanks Cuba for 'great gesture of solidarity' in rescuing passengers from coronavirus cruise ship - A spokesperson for cruise operator Fred Olsen said at the time that "no other Caribbean ports were willing to accept the ship because of local sensitivities towards COVID-19 coronavirus".
A zoo has been trying to get two pandas to mate for 10 years. When coronavirus shut the zoo down, the pandas finally did
Bill Gates calls Taiwan's handling of coronavirus 'exemplary'
Zoom banned by Taiwan's government over China security fears
A town mayor has been expelled from the Labour party for saying Boris Johnson ‘completely deserves’ to have coronavirus.
Sweden is facing increasing pressure to impose a coronavirus lockdown after the number of deaths rose by nearly 20 per cent in a day to 477. The government has so far resisted calls to shut down pubs, restaurants, offices and

In [10]:
# Defining a function that gets the data from reddit
from time import sleep

def get_links(subreddit, token, n_pages=5):
    stories = []
    after = None
    for page_number in range(n_pages):
        # Sleep before making calls to avoid going over the API limit
        sleep(2)
        # Setup headers and make call, just like in the login function
        headers = {"Authorization": f"bearer {token['access_token']}", "User-Agent": USER_AGENT}
        url = f"https://oauth.reddit.com/r/{subreddit}?limit=100"
        if after:
            # Append cursor for next page, if we have one
            url += f"&after={after}"
        response = requests.get(url, headers=headers)
        result = response.json()
        # Get the new cursor for the next loop
        after = result['data']['after']
        # Add all os the news items to our stories list
        for story in result['data']['children']:
            stories.append((story['data']['title'], story['data']['url'], story['data']['score']))
    return stories

In [11]:
# Calling the stories function
stories = get_links("worldnews", token)
print(len(stories))

501


In [12]:
# Setting up the data folder
import os 

data_folder = os.path.join(os.path.expanduser("~"), "data", "raw")

In [13]:
# Importing hashlib library in order to use md5 hashing to create unique filenames for our articles by hashing the URL
import hashlib

In [14]:
# Iterating through each story, downloading website and saving the results to a file
number_errors = 0

for title, url, score in stories:
    if number_errors == 100:
        break
    output_filename = hashlib.md5(url.encode()).hexdigest() 
    fullpath = os.path.join(data_folder, output_filename + ".txt")
    try: 
        response = requests.get(url) 
        data = response.text 
        with open(fullpath, 'w') as outf: 
            outf.write(data)
    except Exception as e:
        number_errors += 1
        print(e)
        # You can use this to view the errors, if you are getting too many:
        # raise
        print(number_errors)

0
0
0
0
'charmap' codec can't encode character '\u6f22' in position 8427: character maps to <undefined>
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
'charmap' codec can't encode character '\x80' in position 62860: character maps to <undefined>
2
2
2
2
'charmap' codec can't encode character '\u2028' in position 213307: character maps to <undefined>
3
3
3
'charmap' codec can't encode characters in position 37364-37365: character maps to <undefined>
4
4
'charmap' codec can't encode characters in position 51619-51622: character maps to <undefined>
5
5
5
5
5
5
5
5
5
5
5
5
'charmap' codec can't encode character '\u011f' in position 1709: character maps to <undefined>
6
6
6
6
6
'charmap' codec can't encode character '\u2032' in position 11235: character maps to <undefined>
7
7
7
7
'charmap' codec can't encode characters in position 301-304: character maps to <undefined>
8
'charmap' codec can't encode character '\ufeff' in position 101125: character maps to <undefined>
9
'charmap' codec can't encode char

In [15]:
# Checking the number of errors obtained
print(f"Number of errors: {number_errors}")

Number of errors: 100


In [17]:
# Getting a list of each of the filenames in our raw subfolder
filenames = [os.path.join(data_folder, filename) for filename in os.listdir(data_folder)]

In [18]:
# Setting up the data folder for the text-only versions that we will extract
text_output_folder = os.path.join(os.path.expanduser("~"), "data", "textonly")

In [19]:
# Code that will extract the text from files
from lxml import etree

skip_node_types = ["script", "head", "style", etree.Comment]

In [20]:
# Defining a function that parses an HTML file into an lxml etree
parser = etree.HTMLParser()

def get_text_from_file(filename):
    with open(filename) as inf:
        html_tree = etree.parse(inf, parser)
    return get_text_from_node(html_tree.getroot())

In [47]:
# Defining a function that checks if the text is at least 100 characters long
def get_text_from_node(node):
    try:
        if len(node) == 0: 
            # No children, just return text from this item
            if node.text:
                return node.text 
            else:
                return ""
        else:
            # This node has children, return the text from it:
            results = (get_text_from_node(child)
                    for child in node
                    if child.tag not in skip_node_types)
        result = str.join("\n", (r for r in results if len(r) > 1))
        if len(result) >= 100:
            return result
        else:
            return ""
    except Exception as e:
        pass

In [48]:
# Iterating through the raw HTML pages and saving results to the textonly subfolder
for filename in os.listdir(data_folder):
    try:
        text = get_text_from_file(os.path.join(data_folder, filename)) 
        with open(os.path.join(text_output_folder, filename), 'w') as outf: 
            outf.write(text)
    except:
        continue

In [49]:
# Importing k-means algorithm and CountVectorizer
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer

In [50]:
# Setting up the pipeline for the analysis
from sklearn.pipeline import Pipeline

n_clusters = 10
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)), ('clusterer', KMeans(n_clusters=n_clusters))])

In [51]:
# Fitting and predicting the pipeline
documents = [open(os.path.join(text_output_folder, filename)).read()
                for filename in os.listdir(text_output_folder)]
pipeline.fit(documents)
labels = pipeline.predict(documents)

In [52]:
# Checking the number of samples in each cluster
from collections import Counter

c = Counter(labels)
for cluster_number in range(n_clusters):
    print(f"Cluster {cluster_number} contains {c[cluster_number]} samples")

Cluster 0 contains 257 samples
Cluster 1 contains 13 samples
Cluster 2 contains 1 samples
Cluster 3 contains 1 samples
Cluster 4 contains 5 samples
Cluster 5 contains 6 samples
Cluster 6 contains 3 samples
Cluster 7 contains 30 samples
Cluster 8 contains 2 samples
Cluster 9 contains 4 samples


In [53]:
# Checking algorithm result
pipeline.named_steps['clusterer'].inertia_

146.30843263452965

In [54]:
# Getting a list of scores for each n_cluster value between 2 and 20
inertia_scores = []
n_cluster_values = list(range(2, 20))
for n_clusters in n_cluster_values:
    cur_inertia_scores = []
    X = TfidfVectorizer(max_df=0.4).fit_transform(documents)
    for i in range(10):
        km = KMeans(n_clusters=n_clusters).fit(X)
        cur_inertia_scores.append(km.inertia_)
        inertia_scores.append(cur_inertia_scores)

In [66]:
# Rerunning algorithm according to the plot result
n_clusters = 6
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)), ('clusterer', KMeans(n_clusters=n_clusters))])
pipeline.fit(documents)
labels = pipeline.predict(documents)

In [67]:
# Extracting the term list from our feature extraction step
terms = pipeline.named_steps['feature_extraction'].get_feature_names()
c = Counter(labels)

In [68]:
# Iterate over each cluster and print the size of cluster and most important terms for this cluster
for cluster_number in range(n_clusters):
    print(f"Cluster {cluster_number} contains {c[cluster_number]} samples")
    print(f"  Most important terms")
    centroid = pipeline.named_steps['clusterer'].cluster_centers_[cluster_number]
    most_important = centroid.argsort()
    for i in range(5):
        term_index = most_important[-(i+1)]
        print(f"  {i+1}) {terms[term_index]} (score: {centroid[term_index]:.4f})")

Cluster 0 contains 268 samples
  Most important terms
  1) he (score: 0.0238)
  2) coronavirus (score: 0.0209)
  3) was (score: 0.0184)
  4) will (score: 0.0176)
  5) they (score: 0.0175)
Cluster 1 contains 1 samples
  Most important terms
  1) mr (score: 0.3122)
  2) government (score: 0.2082)
  3) he (score: 0.2017)
  4) care (score: 0.1731)
  5) decisions (score: 0.1707)
Cluster 2 contains 13 samples
  Most important terms
  1) cache (score: 0.4414)
  2) found (score: 0.3217)
  3) 404 (score: 0.2207)
  4) server (score: 0.2207)
  5) mediation (score: 0.2207)
Cluster 3 contains 5 samples
  Most important terms
  1) detected (score: 0.2977)
  2) robot (score: 0.2977)
  3) unusual (score: 0.2895)
  4) computer (score: 0.2895)
  5) click (score: 0.2822)
Cluster 4 contains 32 samples
  Most important terms
  1) tablet (score: 0.2705)
  2) browser (score: 0.2030)
  3) landscape (score: 0.1503)
  4) medium (score: 0.1476)
  5) portrait (score: 0.1202)
Cluster 5 contains 3 samples
  Most im

In [71]:
# Ignoring some words
function_words = ["a", "able", "aboard", "about", "above", "absent",
                "according" , "accordingly", "across", "after", "against",
                "ahead", "albeit", "all", "along", "alongside", "although",
                "am", "amid", "amidst", "among", "amongst", "amount", "an",
                "and", "another", "anti", "any", "anybody", "anyone",
                "anything", "are", "around", "as", "aside", "astraddle",
                "astride", "at", "away", "bar", "barring", "be", "because",
                "been", "before", "behind", "being", "below", "beneath",
                "beside", "besides", "better", "between", "beyond", "bit",
                "both", "but", "by", "can", "certain", "circa", "close",
                "concerning", "consequently", "considering", "could",
                "couple", "dare", "deal", "despite", "down", "due", "during",
                "each", "eight", "eighth", "either", "enough", "every",
                "everybody", "everyone", "everything", "except", "excepting",
                "excluding", "failing", "few", "fewer", "fifth", "first",
                "five", "following", "for", "four", "fourth", "from", "front",
                "given", "good", "great", "had", "half", "have", "he",
                "heaps", "hence", "her", "hers", "herself", "him", "himself",
                "his", "however", "i", "if", "in", "including", "inside",
                "instead", "into", "is", "it", "its", "itself", "keeping",
                "lack", "less", "like", "little", "loads", "lots", "majority",
                "many", "masses", "may", "me", "might", "mine", "minority",
                "minus", "more", "most", "much", "must", "my", "myself",
                "near", "need", "neither", "nevertheless", "next", "nine",
                "ninth", "no", "nobody", "none", "nor", "nothing",
                "notwithstanding", "number", "numbers", "of", "off", "on",
                "once", "one", "onto", "opposite", "or", "other", "ought",
                "our", "ours", "ourselves", "out", "outside", "over", "part",
                "past", "pending", "per", "pertaining", "place", "plenty",
                "plethora", "plus", "quantities", "quantity", "quarter",
                "regarding", "remainder", "respecting", "rest", "round",
                "save", "saving", "second", "seven", "seventh", "several",
                "shall", "she", "should", "similar", "since", "six", "sixth",
                "so", "some", "somebody", "someone", "something", "spite",
                "such", "ten", "tenth", "than", "thanks", "that", "the",
                "their", "theirs", "them", "themselves", "then", "thence",
                "therefore", "these", "they", "third", "this", "those",
                "though", "three", "through", "throughout", "thru", "thus",
                "till", "time", "to", "tons", "top", "toward", "towards",
                "two", "under", "underneath", "unless", "unlike", "until",
                "unto", "up", "upon", "us", "used", "various", "versus",
                "via", "view", "wanting", "was", "we", "were", "what",
                "whatever", "when", "whenever", "where", "whereas",
                "wherever", "whether", "which", "whichever", "while",
                "whilst", "who", "whoever", "whole", "whom", "whomever",
                "whose", "will", "with", "within", "without", "would", "yet",
                "you", "your", "yours", "yourself", "yourselves"]

pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4, stop_words=function_words)), ('clusterer', KMeans(n_clusters=n_clusters))])
pipeline.fit(documents)

Pipeline(memory=None,
         steps=[('feature_extraction',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=0.4, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=['a', 'able', 'aboard', 'about',
                                             'above', 'absent'...
                                             'and', 'another', 'anti', 'any',
                                             'anybody', 'anyone', ...],
                                 strip_accents=None, sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                              

In [72]:
# Creating a dataset with a lower number of features by taking the distance to each centroid as a feature
X = pipeline.transform(documents)

In [75]:
# Creating a co-association matrix from an array of labels
from scipy.sparse import csr_matrix

def create_coassociation_matrix(labels):
    rows = []
    cols = []
    unique_labels = set(labels)
    for label in unique_labels:
        indices = np.where(labels == label)[0]
        for index1 in indices:
            for index2 in indices:
                rows.append(index1)
                cols.append(index2)
    data = np.ones((len(rows),))
    return csr_matrix((data, (rows, cols)), dtype='float')

In [76]:
# Calling the function and getting the co-association matrix
C = create_coassociation_matrix(labels)
print(C)

(0, 0)	1.0
  (0, 1)	1.0
  (0, 2)	1.0
  (0, 3)	1.0
  (0, 4)	1.0
  (0, 5)	1.0
  (0, 6)	1.0
  (0, 7)	1.0
  (0, 8)	1.0
  (0, 9)	1.0
  (0, 11)	1.0
  (0, 12)	1.0
  (0, 15)	1.0
  (0, 16)	1.0
  (0, 17)	1.0
  (0, 18)	1.0
  (0, 19)	1.0
  (0, 20)	1.0
  (0, 21)	1.0
  (0, 22)	1.0
  (0, 23)	1.0
  (0, 24)	1.0
  (0, 25)	1.0
  (0, 27)	1.0
  (0, 28)	1.0
  :	:
  (320, 293)	1.0
  (320, 294)	1.0
  (320, 295)	1.0
  (320, 296)	1.0
  (320, 297)	1.0
  (320, 298)	1.0
  (320, 299)	1.0
  (320, 300)	1.0
  (320, 301)	1.0
  (320, 302)	1.0
  (320, 303)	1.0
  (320, 305)	1.0
  (320, 306)	1.0
  (320, 309)	1.0
  (320, 310)	1.0
  (320, 311)	1.0
  (320, 312)	1.0
  (320, 313)	1.0
  (320, 315)	1.0
  (320, 316)	1.0
  (320, 318)	1.0
  (320, 320)	1.0
  (321, 107)	1.0
  (321, 319)	1.0
  (321, 321)	1.0


In [77]:
# Computing the Minimum Spanning Tree
from scipy.sparse.csgraph import minimum_spanning_tree

mst = minimum_spanning_tree(C)

In [78]:
mst = minimum_spanning_tree(-C)

In [79]:
# Removing any node with a weight less than a predefined threshold
pipeline.fit(documents)
labels2 = pipeline.predict(documents)
C2 = create_coassociation_matrix(labels2)
C_sum = (C+C2)/2
mst = minimum_spanning_tree(-C_sum)
mst.data[mst.data > -1] = 0

In [80]:
from scipy.sparse.csgraph import connected_components 
number_of_clusters, labels = connected_components(mst)
print(f"Number of clusters: {number_of_clusters}")

Number of clusters: 3


In [81]:
from sklearn.base import BaseEstimator, ClusterMixin 
class EAC(BaseEstimator, ClusterMixin):
    def __init__(self, n_clusterings=10, cut_threshold=0.5, n_clusters_range=(3, 10)): 
        self.n_clusterings = n_clusterings
        self.cut_threshold = cut_threshold
        self.n_clusters_range = n_clusters_range

    def fit(self, X, y=None):
        C = sum((create_coassociation_matrix(self._single_clustering(X))
                 for i in range(self.n_clusterings)))
        mst = minimum_spanning_tree(-C)
        mst.data[mst.data > -self.cut_threshold] = 0
        mst.eliminate_zeros()
        self.n_components, self.labels_ = connected_components(mst)
        return self
 
    def _single_clustering(self, X):
        n_clusters = np.random.randint(*self.n_clusters_range)
        km = KMeans(n_clusters=n_clusters)
        return km.fit_predict(X)
 
    def fit_predict(self, X):
        self.fit(X)
        return self.labels_

In [82]:
pipeline = Pipeline([('feature_extraction', TfidfVectorizer(max_df=0.4)),
                     ('clusterer', EAC()) ])

In [83]:
vec = TfidfVectorizer(max_df=0.4) 
X = vec.fit_transform(documents)

In [84]:
from sklearn.cluster import MiniBatchKMeans 
mbkm = MiniBatchKMeans(random_state=14, n_clusters=3)

In [85]:
batch_size = 10 
for iteration in range(int(X.shape[0] / batch_size)): 
    start = batch_size * iteration 
    end = batch_size * (iteration + 1) 
    mbkm.partial_fit(X[start:end])

In [87]:
labels = mbkm.predict(X)
print(labels)

[2 2 2 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 1 2 2 1 1 2 2 1 2 1 2 2 2 1 2 1 2 1 1
 2 2 1 2 2 1 2 1 2 1 2 2 1 2 1 2 1 2 2 2 1 1 2 1 2 1 1 2 2 2 1 1 1 2 1 2 2
 1 1 1 1 1 1 2 2 2 1 1 1 1 2 2 1 2 1 2 2 1 2 2 2 2 1 1 2 2 2 1 1 1 2 1 2 1
 2 2 2 1 2 1 2 2 2 1 1 2 1 2 2 2 2 1 2 2 2 2 2 2 2 2 1 2 1 2 2 1 2 2 2 2 2
 2 2 1 1 2 2 1 1 1 2 1 1 1 1 2 2 1 2 1 2 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 1 1
 1 1 2 2 2 2 2 2 2 2 1 2 1 2 1 1 2 1 2 1 1 1 1 0 2 2 2 1 1 1 2 1 1 2 2 2 2
 1 1 2 1 1 2 2 2 1 2 1 2 2 1 2 2 2 2 2 1 1 2 2 2 1 2 2 2 1 2 2 2 2 1 1 2 1
 1 1 1 2 2 1 2 2 1 1 1 2 1 1 2 2 1 1 2 2 1 1 1 1 2 2 2 1 1 2 2 1 2 1 1 1 2
 2 1 2 1 1 1 2 1 2 1 1 2 2 2 2 1 1 2 2 2 2 2 1 2 1 2]


In [88]:
class PartialFitPipeline(Pipeline):
    def partial_fit(self, X, y=None):
        Xt = X
        for name, transform in self.steps[:-1]:
            Xt = transform.transform(Xt)
        return self.steps[-1][1].partial_fit(Xt, y=y)

In [90]:
from sklearn.feature_extraction.text import HashingVectorizer

pipeline = PartialFitPipeline([('feature_extraction', HashingVectorizer()), ('clusterer', MiniBatchKMeans(random_state=14, n_clusters=3))])
batch_size = 10
for iteration in range(int(len(documents) / batch_size)): 
    start = batch_size * iteration 
    end = batch_size * (iteration + 1)
    pipeline.partial_fit(documents[start:end]) 
labels = pipeline.predict(documents)
print(labels)

[0 0 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 1 0 0 1 0 1 0 0 0 1 0 1 0 1 1
 0 0 1 0 0 1 0 1 0 1 0 0 1 0 1 0 1 1 0 0 1 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0
 1 1 1 1 1 1 0 0 1 1 1 1 1 0 0 1 0 1 0 0 1 0 0 0 0 1 1 0 0 0 1 1 1 0 1 0 1
 0 0 1 1 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0
 0 0 1 1 0 0 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 0 0 0 0 1 0 0 0 1 1
 1 1 0 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 1 1 2 0 0 0 1 1 1 0 1 1 1 0 0 0
 1 1 0 1 1 0 0 0 1 1 1 0 1 1 0 0 0 0 0 1 1 0 0 1 1 0 1 0 0 0 0 0 0 1 1 1 1
 1 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 1 1 0 0 1 1 1 1 0 0 0 1 1 0 0 1 0 1 1 1 0
 0 1 0 1 1 1 0 1 1 1 1 1 0 0 0 1 1 0 0 0 0 1 1 0 1 0]
